<a href="https://colab.research.google.com/github/vrangayyan6/GenAI/blob/main/Cerebras_Multi_agent_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Agentic Workflow with Cerebras, Google, LangChain and LangGraph

https://github.com/Cerebras/inference-examples/tree/main/multi-ai-workflow

# Setting up Google Search API Credentials

Before running the main code, you need to set up your Google Search API credentials:

## Step 1: Get a Google API Key
1. Go to the [Google Cloud Console](https://console.cloud.google.com/)
2. Create a new project or select an existing one
3. Enable the "Custom Search API" for your project
4. Go to "Credentials" and create an API key

## Step 2: Create a Programmable Search Engine
1. Go to [Programmable Search Engine](https://programmablesearchengine.google.com/about/)
2. Click "Create a Programmable Search Engine"
3. Configure your search engine (you can search the entire web)
4. After creation, find your "Search engine ID" (also called CSE ID)


In [1]:
!pip install -q langchain_cerebras langchain_community langgraph langchain langchain-core langsmith langchain_experimental cerebras_cloud_sdk langchain-google-community google-search-results


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
import os
import time
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph import END
from langchain_cerebras import ChatCerebras
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_google_community import GoogleSearchRun
from google.colab import userdata
from IPython.display import Markdown

In [3]:
# Add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Make sure these are set BEFORE creating any search wrappers
os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLESEARCH_API_KEY')

final_result = []

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

api_key = userdata.get('CEREBRAS_API_KEY')

class State(TypedDict):
    query: Annotated[list, add_messages]
    research: Annotated[list, add_messages]
    content: str
    content_ready: bool
    iteration_count: int
    # Counter for iterations

# Initialize ChatCerebras instance for language model
llm = ChatCerebras(api_key=api_key, model="llama-4-scout-17b-16e-instruct")  # llama-3.3-70b

# The GoogleSearchRun class is already implemented in langchain_community.tools

class ResearchAgent:
    def format_search(self, query: str) -> str:
        prompt = (
            "You are an expert at optimizing search queries for Google. "
            "Your task is to take a given query and return an optimized version of it, making it more likely to yield relevant results. "
            "Do not include any explanations or extra text, only the optimized query.\n\n"
            "Example:\n"
            "Original: best laptop 2023 for programming\n"
            "Optimized: top laptops 2023 for coding\n\n"
            "Example:\n"
            "Original: how to train a puppy not to bite\n"
            "Optimized: puppy training tips to prevent biting\n\n"
            "Now optimize the following query:\n"
            f"Original: {query}\n"
            "Optimized:"
        )

        response = llm.invoke(prompt)
        return response.content

    def search(self, state: State):
        # Initialize the Google Search API wrapper first
        search_wrapper = GoogleSearchAPIWrapper(k=10)
        # Pass the wrapper to GoogleSearchRun
        search = GoogleSearchRun(api_wrapper=search_wrapper)

        start_time = time.perf_counter()
        optimized_query = self.format_search(state.get('query', "")[-1].content)
        end_time = time.perf_counter()

        results = search.invoke(optimized_query)

        state["optimized_query"] = optimized_query

        final_result.append({"subheader": f"Research Iteration", "content": [results], "time": end_time - start_time})
        print("Search results: ", results)
        return {"research": results}

class EditorAgent:
    def evaluate_research(self, state: State):
        query = '\n'.join(message.content for message in state.get("query"))
        research = '\n'.join(message.content for message in state.get("research"))

        iteration_count = state.get("iteration_count", 1)

        if iteration_count is None:
            iteration_count = 1

        if iteration_count >= 11:
            return {"content_ready": True}

        prompt = (
            "You are an expert editor. Your task is to evaluate the research based on the query. "
            "If the information is sufficient to create a comprehensive and accurate blog post, respond with 'sufficient'. "
            "If the information is not sufficient, respond with 'insufficient' and provide a new, creative query suggestion to improve the results. "
            "If the research results appear repetitive or not diverse enough, think about a very different kind of question that could yield more varied and relevant information. "
            "Consider the depth, relevance, and completeness of the information when making your decision.\n\n"
            "Example 1:\n"
            "Used queries: What are the benefits of a Mediterranean diet?\n"
            "Research: The Mediterranean diet includes fruits, vegetables, whole grains, and healthy fats.\n"
            "Evaluation: Insufficient\n"
            "New query: Detailed health benefits of a Mediterranean diet\n\n"
            "Example 2:\n"
            "Used queries: How does solar power work?\n"
            "Research: Solar power works by converting sunlight into electricity using photovoltaic cells.\n"
            "Evaluation: Sufficient\n\n"
            "Example 3:\n"
            "Used queries: Effects of climate change on polar bears?\n"
            "Research: Climate change is reducing sea ice, affecting polar bear habitats.\n"
            "Evaluation: Insufficient\n"
            "New query: How are polar bears adapting to the loss of sea ice due to climate change?\n\n"
            "Now evaluate the following:\n"
            f"Used queries: {query}\n"
            f"Research: {research}\n\n"
            "Evaluation (sufficient/insufficient):\n"
            "New query (if insufficient):"
        )

        start_time = time.perf_counter()
        response = llm.invoke(prompt)
        end_time = time.perf_counter()

        evaluation = response.content.strip()

        final_result.append({"subheader": f"Editor Evaluation Iteration", "content": evaluation, "time": end_time - start_time})

        if "new query:" in evaluation.lower():
            new_query = evaluation.split("New query:", 1)[-1].strip()
            return {"query": [new_query], "iteration_count": iteration_count + 1, "evaluation": evaluation}
        else:
            return {"content_ready": True, "evaluation": evaluation}

class WriterAgent:
    def write_blogpost(self, state: State):
        query = state.get("query")[0].content
        research = '\n'.join(message.content for message in state.get("research"))

        prompt = (
            "You are an expert blog post writer. Your task is to take a given query and context, and write a comprehensive, engaging, and informative short blog post about it. "
            "Make sure to include an introduction, main body with detailed information, and a conclusion. Use a friendly and accessible tone, and ensure the content is well-structured and easy to read.\n\n"
            f"Query: {query}\n\n"
            f"Context:\n{research}\n\n"
            "Write a detailed and engaging blog post based on the above query and context."
        )

        response  = llm.invoke(prompt)

        return {"content": response.content}

# Initialize the StateGraph
graph = StateGraph(State)

graph.add_node("search_agent", ResearchAgent().search)
graph.add_node("writer_agent", WriterAgent().write_blogpost)
graph.add_node("editor_agent", EditorAgent().evaluate_research)

graph.set_entry_point("search_agent")

graph.add_edge("search_agent", "editor_agent")

graph.add_conditional_edges(
    "editor_agent",
    lambda state: "accept" if state.get("content_ready") else "revise",
    {
        "accept": "writer_agent",
        "revise": "search_agent"
    }
)

graph.add_edge("writer_agent", END)

graph = graph.compile()

def invoke_graph(user_prompt):
  start_time = time.perf_counter()
  blogpost = graph.invoke({"query": user_prompt})
  end_time = time.perf_counter()
  print("\n\n")
  return blogpost["content"]

# Provide your prompt below

In [4]:
user_prompt = "Act as an expert in Financial Services, explain in detail on separately managed accounts"

In [5]:
Markdown(invoke_graph(user_prompt))

Search results:  May 1, 2024 ... Innovation in mutual fund variety really didn't take off until the 1980s and 1990s, when funds became the standard investment choices of defined ... It takes a hefty minimum investment to establish a separately managed account (SMA), but it does offer some distinct advantages. Nov 3, 2022 ... A SMA is a portfolio of securities managed by an investment firm on an investor's behalf. Unlike a mutual fund or Exchange Traded Fund (ETF), the investor ... Jan 3, 2025 ... If you work with a financial advisor who manages your investments, you might have a separately managed account. Here's what that means and ... An SMA is a professionally managed portfolio of individual investments designed to meet a specific objective—one component of your overall investment portfolio ... What is a separately managed account? · Contact your Schwab Financial Consultant or connect with us to learn more. · Ownership has its advantages. · Find the ... What are the benefits of a se

**Separately Managed Accounts: A Comprehensive Guide for Investors**

As an investor, you're likely familiar with traditional investment vehicles like mutual funds and exchange-traded funds (ETFs). However, there's another option that offers a high degree of customization and control: separately managed accounts (SMAs). In this blog post, we'll dive into the world of SMAs, exploring their benefits, how they differ from mutual funds and ETFs, and who they're best suited for.

**What is a Separately Managed Account (SMA)?**

A separately managed account is a portfolio of individual securities managed by an investment firm on an investor's behalf. Unlike mutual funds or ETFs, where investors own shares of the fund, SMAs allow investors to own the individual securities directly. This provides a high degree of transparency and control, as investors can customize their portfolio to meet their specific investment objectives and values.

**Benefits of Separately Managed Accounts**

So, what are the benefits of SMAs? Here are a few:

* **Customization**: SMAs allow investors to establish guidelines for managing their investments in line with their personal beliefs and needs. For instance, you could restrict investment in certain industries or companies that don't align with your values.
* **Tax efficiency**: SMAs can provide greater tax efficiencies than mutual funds or ETFs, as investors can harvest losses and optimize their tax strategy.
* **Transparency**: With an SMA, investors have direct ownership of individual securities, providing a clear picture of their portfolio holdings.
* **Flexibility**: SMAs can be tailored to meet the specific needs of high-net-worth individuals, institutional investors, or family offices.

**How do SMAs differ from Mutual Funds or ETFs?**

There are several key differences between SMAs and traditional investment vehicles like mutual funds and ETFs:

* **Ownership**: With mutual funds or ETFs, investors own shares of the fund, not the individual securities inside. In contrast, SMAs allow investors to own the individual securities directly.
* **Customization**: Mutual funds and ETFs are designed for a broad range of investors, while SMAs can be tailored to meet the specific needs of individual investors.
* **Fees**: SMAs often come with higher management fees compared to mutual funds or ETFs, but offer a high degree of customization and control.

**Who are Separately Managed Accounts best suited for?**

SMAs are typically best suited for:

* **High-net-worth individuals**: SMAs offer a high degree of customization and control, making them an attractive option for high-net-worth individuals who require tailored investment solutions.
* **Institutional investors**: SMAs can be used by institutional investors, such as pension funds or endowments, to meet their specific investment objectives.
* **Family offices**: SMAs can provide family offices with a customized investment solution that meets their unique needs and goals.

**Direct Indexing through SMAs**

Direct indexing is a strategy that involves owning individual stocks that make up a chosen index, rather than investing in a mutual fund or ETF that tracks the same index. SMAs can provide a direct indexing solution, offering greater autonomy, control, and tax advantages over traditional investment vehicles.

**Conclusion**

Separately managed accounts offer a unique combination of customization, control, and tax efficiency that can be attractive to high-net-worth individuals, institutional investors, and family offices. While SMAs come with higher management fees compared to mutual funds or ETFs, they provide a high degree of flexibility and transparency that can be valuable for investors who require tailored investment solutions. As the investment landscape continues to evolve, SMAs are likely to play an increasingly important role in the world of investment management.

Whether you're a seasoned investor or just starting to explore your investment options, it's essential to understand the benefits and drawbacks of SMAs. By considering your individual needs and goals, you can determine whether a separately managed account is right for you.